<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liars in the Miller-Rabin primality test by Arjun Yadav
## All of the code and results are available on [GitHub](https://github.com/y-arjun-y/liars-miller-rabin). Inspired by [Numberphile](https://www.youtube.com/watch?v=_MscGSN5J6o).

## Implementation of the Miller-Rabin primality test in Python

In [2]:
'''
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python with slight changes by me to support a custom witness (a). 
Function and its formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of false means n in not prime (i.e. composite).
A return value of true means n is very likely to be a prime (likelihood depending on the number of rounds).
'''

def miller_rabin_base(n, a):
  # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1
    
    assert(2**s * d == n - 1)

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials rounds
    num_trials = 10

    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

## Supporting multiple witnesses in the Miller-Rabin primality test

In [3]:
'''
Returns false if any one of the witness returns false and returns true if all witnesses return true.
'''

def miller_rabin(num, witness_list):
  result = []
  
  for i in range(len(witness_list)):
    result.append(miller_rabin_base(num, witness_list[i]))
  
  if False in result:
    return False
  
  return True

## Simple prime number test

In [4]:
'''
Returns whether a number is prime (true) or not (false).
Credit: https://en.wikipedia.org/wiki/Primality_test#Python
'''

def actual_prime(n):
    if n <= 3:
        return n > 1
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i ** 2 <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

## Finding the worst witness in the Miller-Rabin primality test (for `n` in a certain range, 1 witness)

In [29]:
from tqdm import tqdm

DEFENDANT_RANGE = 100
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE

results = {}

print([i for i in range(WITNESS_RANGE)])

# trial loop function
def trial_loop(defendants, witness):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin_base(defendant, witness) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(range(WITNESS_RANGE)):
  results[i] = trial_loop(DEFENDANTS, i)

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:100]:
  print(i)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98]


100%|██████████| 99/99 [00:00<00:00, 723.60it/s]

(1, 24)
(0, 21)
(74, 5)
(76, 5)
(82, 5)
(38, 4)
(62, 4)
(68, 4)
(79, 4)
(18, 3)
(22, 3)
(26, 3)
(29, 3)
(31, 3)
(34, 3)
(43, 3)
(44, 3)
(46, 3)
(47, 3)
(53, 3)
(57, 3)
(64, 3)
(67, 3)
(69, 3)
(80, 3)
(83, 3)
(86, 3)
(89, 3)
(91, 3)
(92, 3)
(94, 3)
(98, 3)
(13, 2)
(16, 2)
(17, 2)
(19, 2)
(32, 2)
(41, 2)
(50, 2)
(51, 2)
(52, 2)
(55, 2)
(56, 2)
(58, 2)
(59, 2)
(61, 2)
(65, 2)
(66, 2)
(71, 2)
(73, 2)
(77, 2)
(78, 2)
(85, 2)
(88, 2)
(93, 2)
(97, 2)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(14, 1)
(20, 1)
(23, 1)
(24, 1)
(28, 1)
(30, 1)
(33, 1)
(36, 1)
(37, 1)
(39, 1)
(40, 1)
(48, 1)
(49, 1)
(54, 1)
(70, 1)
(72, 1)
(75, 1)
(81, 1)
(84, 1)
(87, 1)
(90, 1)
(95, 1)
(96, 1)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(15, 0)
(21, 0)
(25, 0)
(27, 0)
(35, 0)
(42, 0)
(45, 0)
(60, 0)
(63, 0)


## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 2 witnesses)

In [30]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 100
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:100]:
  print(i)

100%|██████████| 9801/9801 [00:16<00:00, 594.96it/s]

((1, 1), 24)
((0, 0), 21)
((0, 1), 21)
((0, 2), 21)
((0, 3), 21)
((0, 4), 21)
((0, 5), 21)
((0, 6), 21)
((0, 7), 21)
((0, 8), 21)
((0, 9), 21)
((0, 10), 21)
((0, 11), 21)
((0, 12), 21)
((0, 13), 21)
((0, 14), 21)
((0, 15), 21)
((0, 16), 21)
((0, 17), 21)
((0, 18), 21)
((0, 19), 21)
((0, 20), 21)
((0, 21), 21)
((0, 22), 21)
((0, 23), 21)
((0, 24), 21)
((0, 25), 21)
((0, 26), 21)
((0, 27), 21)
((0, 28), 21)
((0, 29), 21)
((0, 30), 21)
((0, 31), 21)
((0, 32), 21)
((0, 33), 21)
((0, 34), 21)
((0, 35), 21)
((0, 36), 21)
((0, 37), 21)
((0, 38), 21)
((0, 39), 21)
((0, 40), 21)
((0, 41), 21)
((0, 42), 21)
((0, 43), 21)
((0, 44), 21)
((0, 45), 21)
((0, 46), 21)
((0, 47), 21)
((0, 48), 21)
((0, 49), 21)
((0, 50), 21)
((0, 51), 21)
((0, 52), 21)
((0, 53), 21)
((0, 54), 21)
((0, 55), 21)
((0, 56), 21)
((0, 57), 21)
((0, 58), 21)
((0, 59), 21)
((0, 60), 21)
((0, 61), 21)
((0, 62), 21)
((0, 63), 21)
((0, 64), 21)
((0, 65), 21)
((0, 66), 21)
((0, 67), 21)
((0, 68), 21)
((0, 69), 21)
((0, 70), 21)
((0

## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 3 witnesses)

In [19]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 25
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:100]:
  print(i)

100%|██████████| 15625/15625 [00:05<00:00, 2960.28it/s]

((0, 0, 0), 5)
((0, 0, 1), 5)
((0, 0, 2), 5)
((0, 0, 3), 5)
((0, 0, 4), 5)
((0, 0, 5), 5)
((0, 0, 6), 5)
((0, 0, 7), 5)
((0, 0, 8), 5)
((0, 0, 9), 5)
((0, 0, 10), 5)
((0, 0, 11), 5)
((0, 0, 12), 5)
((0, 0, 13), 5)
((0, 0, 14), 5)
((0, 0, 15), 5)
((0, 0, 16), 5)
((0, 0, 17), 5)
((0, 0, 18), 5)
((0, 0, 19), 5)
((0, 0, 20), 5)
((0, 0, 21), 5)
((0, 0, 22), 5)
((0, 0, 23), 5)
((0, 0, 24), 5)
((0, 1, 0), 5)
((0, 1, 1), 5)
((0, 1, 2), 5)
((0, 1, 3), 5)
((0, 1, 4), 5)
((0, 1, 5), 5)
((0, 1, 6), 5)
((0, 1, 7), 5)
((0, 1, 8), 5)
((0, 1, 9), 5)
((0, 1, 10), 5)
((0, 1, 11), 5)
((0, 1, 12), 5)
((0, 1, 13), 5)
((0, 1, 14), 5)
((0, 1, 15), 5)
((0, 1, 16), 5)
((0, 1, 17), 5)
((0, 1, 18), 5)
((0, 1, 19), 5)
((0, 1, 20), 5)
((0, 1, 21), 5)
((0, 1, 22), 5)
((0, 1, 23), 5)
((0, 1, 24), 5)
((0, 2, 0), 5)
((0, 2, 1), 5)
((0, 2, 2), 5)
((0, 2, 3), 5)
((0, 2, 4), 5)
((0, 2, 5), 5)
((0, 2, 6), 5)
((0, 2, 7), 5)
((0, 2, 8), 5)
((0, 2, 9), 5)
((0, 2, 10), 5)
((0, 2, 11), 5)
((0, 2, 12), 5)
((0, 2, 13), 5)
((0, 2

## Finding the worst witnesses in the Miller-Rabin primality test (for `n` in a certain range, 4 witnesses)

In [32]:
from tqdm import tqdm
import itertools

DEFENDANT_RANGE = 12
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1) if i == 2 or i % 2 != 0]
if DEFENDANT_RANGE % 2 == 0:
  WITNESS_RANGE = DEFENDANT_RANGE - 1
else:
  WITNESS_RANGE = DEFENDANT_RANGE
WITNESSES = list(itertools.product([i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)], [i for i in range(WITNESS_RANGE)]))

results = {}

# trial loop function
def trial_loop(defendants, witnesses):
  lie_count = 0

  for defendant in defendants:
      if miller_rabin(defendant, [witnesses[0], witnesses[1]]) != actual_prime(defendant):
        lie_count += 1
  
  return lie_count

for i in tqdm(WITNESSES):
  results[i] = trial_loop(DEFENDANTS, (i))

# finding top 100 liars
for i in sorted(results.items(), key=lambda x: x[1], reverse=True)[:100]:
  print(i)

100%|██████████| 14641/14641 [00:00<00:00, 15976.39it/s]

((0, 0, 0, 0), 1)
((0, 0, 0, 1), 1)
((0, 0, 0, 2), 1)
((0, 0, 0, 3), 1)
((0, 0, 0, 4), 1)
((0, 0, 0, 5), 1)
((0, 0, 0, 6), 1)
((0, 0, 0, 7), 1)
((0, 0, 0, 8), 1)
((0, 0, 0, 9), 1)
((0, 0, 0, 10), 1)
((0, 0, 1, 0), 1)
((0, 0, 1, 1), 1)
((0, 0, 1, 2), 1)
((0, 0, 1, 3), 1)
((0, 0, 1, 4), 1)
((0, 0, 1, 5), 1)
((0, 0, 1, 6), 1)
((0, 0, 1, 7), 1)
((0, 0, 1, 8), 1)
((0, 0, 1, 9), 1)
((0, 0, 1, 10), 1)
((0, 0, 2, 0), 1)
((0, 0, 2, 1), 1)
((0, 0, 2, 2), 1)
((0, 0, 2, 3), 1)
((0, 0, 2, 4), 1)
((0, 0, 2, 5), 1)
((0, 0, 2, 6), 1)
((0, 0, 2, 7), 1)
((0, 0, 2, 8), 1)
((0, 0, 2, 9), 1)
((0, 0, 2, 10), 1)
((0, 0, 3, 0), 1)
((0, 0, 3, 1), 1)
((0, 0, 3, 2), 1)
((0, 0, 3, 3), 1)
((0, 0, 3, 4), 1)
((0, 0, 3, 5), 1)
((0, 0, 3, 6), 1)
((0, 0, 3, 7), 1)
((0, 0, 3, 8), 1)
((0, 0, 3, 9), 1)
((0, 0, 3, 10), 1)
((0, 0, 4, 0), 1)
((0, 0, 4, 1), 1)
((0, 0, 4, 2), 1)
((0, 0, 4, 3), 1)
((0, 0, 4, 4), 1)
((0, 0, 4, 5), 1)
((0, 0, 4, 6), 1)
((0, 0, 4, 7), 1)
((0, 0, 4, 8), 1)
((0, 0, 4, 9), 1)
((0, 0, 4, 10), 1)
((0, 